In [1]:
from model import SqueezeNet

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "/home/hyz/.virtualenvs/myenv/local/lib/python2.7/site-packages/tensorflow/python/pywrap_tensorflow.py", line 41, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/hyz/.virtualenvs/myenv/local/lib/python2.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/hyz/.virtualenvs/myenv/local/lib/python2.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
ImportError: libcusolver.so.8.0: cannot open shared object file: No such file or directory


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/install_sources#common_installation_problems

for some common reasons and solutions.  Include the entire stack trace
above this error message w

In [ ]:
m = SqueezeNet(10)
m.load_weights('squeezenet_w-70-1.00.hdf5')

In [ ]:
#m.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train_datagen = ImageDataGenerator()
batch_size = 64
ROOT='/home/hyz/prog/deeplearning/distracted_driving/'
training_set = train_datagen.flow_from_directory(ROOT + 'sub/totrain',
                                                target_size= (224, 224), batch_size=batch_size)

val_datagen = ImageDataGenerator()
val_set = val_datagen.flow_from_directory(ROOT + 'sub/toval',
                                          target_size = (224,224),
                                            batch_size = batch_size,
                                            class_mode = 'categorical')

In [ ]:
filepath="squeezenet_w-{epoch:02d}-{categorical_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='categorical_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
    
m.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy',
            metrics=['categorical_accuracy'])

In [ ]:
!rm -rf mydir
i =training_set.next()
i[0].shape, i[1].shape
import bcolz
params = bcolz.cparams(clevel=8)
with bcolz.carray(i[0], cparams=params, rootdir='mydir'):
    pass

In [ ]:
m.fit_generator(training_set, steps_per_epoch=training_set.samples/ batch_size,
                epochs=5,
               validation_data=val_set,
               validation_steps = val_set.samples/ batch_size,
               callbacks=callbacks_list)

In [ ]:
(14208/96-1)/7/7/3


In [ ]:
submit_datagen = ImageDataGenerator()
b = 256
submit_set = submit_datagen.flow_from_directory(ROOT + 'sub/test', target_size=(224,224),
                                                batch_size=b, class_mode=None, shuffle=False)
   
preds = m.predict_generator(submit_set,
                                  submit_set.samples / b + 1, verbose=1, workers=8
                                  )


In [ ]:
import numpy as np
import csv
preds = np.clip(preds, 0.001, 0.999)
with open('submission2.csv', 'w') as csvfile:
  csvw = csv.writer(csvfile)
  csvw.writerow(['img', 'c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])
  for fn, sc in zip(submit_set.filenames, preds):
      csvw.writerow([fn.split('/')[1]] + list(sc))